# Paper math
> Sanity checking all derivations for the manuscript

In [1]:
import aptr

In [2]:
db = aptr.database.RnaDB()

0.9992494370778083 sequences remain after trimming
0.6346009507130348 sequences remain after filtering


In [ ]:
()